In [1]:
import pandas as pd
import numpy as np

In [2]:
from time import time

In [3]:
%matplotlib inline

In [4]:
df_todos_vinhos = pd.read_csv('winequality.csv', sep=';')

In [6]:
colunas = df_todos_vinhos.columns

In [7]:
colunas = [x.replace(' ', '_') for x in colunas]

In [8]:
df_todos_vinhos.columns = colunas

In [9]:
def trat_alcohol(x):
    try:
        return float(x)
    except:
        return np.NaN

Tratar a coluna alcohol para ser um float, e eliminar os que não puderam ser convertidos

In [10]:
df_todos_vinhos['alcohol'] = df_todos_vinhos['alcohol'].apply(trat_alcohol)

In [11]:
df_td = df_todos_vinhos.dropna()

Adicionar coluna de classificação

In [12]:
def add_class(x):
    # entre 0 e 5 ruim
    if x < 5:
        return 0
    # entre 5 e 7 médio
    elif x < 7:
        return 1
    # maior igual a 7 = bom
    else:
        return 2

In [13]:
df_td['class'] = df_td['quality'].apply(add_class)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


Converter a coluna do tipo de vinho para valor numérico: 0 = White e 1 = Red

In [14]:
df_td['type'] = df_td['type'].apply(lambda x: 0 if x == 'White' else 1)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [15]:
df_out = df_td['class'].copy()

In [16]:
df_val = df_td.drop(columns=['quality', 'class'], axis=1, ).copy()

Criar DataFrames de treino e teste.

In [17]:
from sklearn.model_selection import train_test_split

In [18]:
x_train, x_test, y_train, y_test = train_test_split(df_val, df_out, test_size=0.2)

In [19]:
from sklearn.metrics import accuracy_score, fbeta_score

In [27]:
def evaluate_model(model, sample_size, x_train, x_test, y_train, y_test):
    retorno = {}
    
    # Treinar dados
    op_start = time()
    model = model.fit(x_train[:sample_size], y_train[:sample_size])
    op_end = time()
    
    retorno['training_time'] = op_end - op_start
    
    # Prever o resultado de 15% dos valores usados no treino e nos valores de teste
    _percent = int(sample_size * 0.15)
    op_start = time()
    pred_train = model.predict(x_train[:_percent])
    pred_test = model.predict(x_test)
    op_end = time()
    
    retorno['predicting_time'] = op_end - op_start
    
    # Verificar a acurácia das predições:
    retorno['accuracy_train'] = accuracy_score(y_train[:_percent], pred_train)
    retorno['accuracy_test'] = accuracy_score(y_test, pred_test)
    
    # Verificar o Fbeta das predições:
    retorno['fbeta_train'] = fbeta_score(y_train[:_percent], pred_train, beta=0.5, average='micro')
    retorno['fbeta_test'] = fbeta_score(y_test, pred_test, beta=0.5, average='micro')
    
    print(f'{model.__class__.__name__} treinou com {sample_size} dados de amostra.')
    
    return retorno

Escolhi 3 modelos que possuem formas de prever classificações levando em conta variância e tendência de formas diferentes.
Gauss process classifier, Neural Net, e Random forests.

In [21]:
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.gaussian_process.kernels import RBF

In [22]:
modelo1 = GaussianProcessClassifier(kernel=(1.0 * RBF(1.0)))
modelo2 = MLPClassifier()
modelo3 = RandomForestClassifier()

Dividir as amostras em 3 partes 5%, 25% e 100% dos valores de training

In [23]:
sample5 = int(len(x_train) * 0.05)
sample25 = int(len(x_train) * 0.25)
sample100 = len(x_train)

In [30]:
testes = {}
for md in [modelo1, modelo2, modelo3]:
    md_name = md.__class__.__name__
    testes[md_name] = []
    for sm in [sample5, sample25, sample100]:
        testes[md_name].append(evaluate_model(md, sm, x_train, x_test, y_train, y_test))
        
print(testes)

GaussianProcessClassifier treinou com 258 dados de amostra.
GaussianProcessClassifier treinou com 1291 dados de amostra.
GaussianProcessClassifier treinou com 5165 dados de amostra.


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPClassifier treinou com 258 dados de amostra.
MLPClassifier treinou com 1291 dados de amostra.
MLPClassifier treinou com 5165 dados de amostra.


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier treinou com 258 dados de amostra.
RandomForestClassifier treinou com 1291 dados de amostra.
RandomForestClassifier treinou com 5165 dados de amostra.
{'GaussianProcessClassifier': [{'training_time': 2.1398603916168213, 'predicting_time': 0.07810735702514648, 'accuracy_train': 0.7894736842105263, 'accuracy_test': 0.7469040247678018, 'fbeta_train': 0.7894736842105263, 'fbeta_test': 0.7469040247678018}, {'training_time': 26.88429093360901, 'predicting_time': 0.8435487747192383, 'accuracy_train': 0.7305699481865285, 'accuracy_test': 0.7523219814241486, 'fbeta_train': 0.7305699481865285, 'fbeta_test': 0.7523219814241486}, {'training_time': 2937.434215784073, 'predicting_time': 17.935643911361694, 'accuracy_train': 0.7790697674418605, 'accuracy_test': 0.7755417956656346, 'fbeta_train': 0.7790697674418605, 'fbeta_test': 0.7755417956656346}], 'MLPClassifier': [{'training_time': 0.40615344047546387, 'predicting_time': 0.0, 'accuracy_train': 0.8157894736842105, 'accuracy_t

In [37]:
testes['GaussianProcessClassifier']

[{'train_time': 2.1398603916168213,
  'pred_time': 0.07810735702514648,
  'acc_train': 0.7894736842105263,
  'acc_test': 0.7469040247678018,
  'f_train': 0.7894736842105263,
  'f_test': 0.7469040247678018},
 {'train_time': 26.88429093360901,
  'pred_time': 0.8435487747192383,
  'acc_train': 0.7305699481865285,
  'acc_test': 0.7523219814241486,
  'f_train': 0.7305699481865285,
  'f_test': 0.7523219814241486},
 {'train_time': 2937.434215784073,
  'pred_time': 17.935643911361694,
  'acc_train': 0.7790697674418605,
  'acc_test': 0.7755417956656346,
  'f_train': 0.7790697674418605,
  'f_test': 0.7755417956656346}]

In [38]:
testes['MLPClassifier']

[{'train_time': 0.40615344047546387,
  'pred_time': 0.0,
  'acc_train': 0.8157894736842105,
  'acc_test': 0.7414860681114551,
  'f_train': 0.8157894736842105,
  'f_test': 0.7414860681114551},
 {'train_time': 0.5155057907104492,
  'pred_time': 0.0,
  'acc_train': 0.7461139896373057,
  'acc_test': 0.7523219814241486,
  'f_train': 0.7461139896373056,
  'f_test': 0.7523219814241486},
 {'train_time': 1.6714861392974854,
  'pred_time': 0.015615701675415039,
  'acc_train': 0.7868217054263565,
  'acc_test': 0.771671826625387,
  'f_train': 0.7868217054263565,
  'f_test': 0.771671826625387}]

In [39]:
testes['RandomForestClassifier']

[{'train_time': 0.2030787467956543,
  'pred_time': 0.03167557716369629,
  'acc_train': 1.0,
  'acc_test': 0.7554179566563467,
  'f_train': 1.0,
  'f_test': 0.7554179566563467},
 {'train_time': 0.030862092971801758,
  'pred_time': 0.0,
  'acc_train': 0.9948186528497409,
  'acc_test': 0.7678018575851393,
  'f_train': 0.9948186528497409,
  'f_test': 0.7678018575851393},
 {'train_time': 0.07805395126342773,
  'pred_time': 0.015654563903808594,
  'acc_train': 0.9896640826873385,
  'acc_test': 0.8258513931888545,
  'f_train': 0.9896640826873385,
  'f_test': 0.8258513931888545}]

In [41]:
modelo3.feature_importances_

array([0.00463526, 0.07338717, 0.09219737, 0.07078887, 0.08427217,
       0.08467816, 0.08236391, 0.08507775, 0.10590499, 0.084148  ,
       0.08452304, 0.14802331])

In [42]:
x_train.columns

Index(['type', 'fixed_acidity', 'volatile_acidity', 'citric_acid',
       'residual_sugar', 'chlorides', 'free_sulfur_dioxide',
       'total_sulfur_dioxide', 'density', 'pH', 'sulphates', 'alcohol'],
      dtype='object')